## `genalog.text` module:
This module is responsible for:

- Text alignment
- NER label propagation using text alignment results

`genalog` provides two methods of alignment:
1. `genalog.text.anchor.align_w_anchor()`
1. `genalog.text.alignment.align()`

`align_w_anchor()` implements the Recursive Text Alignment Scheme (RETAS) from the paper [A Fast Alignment Scheme for Automatic OCR Evaluation of Books](https://ieeexplore.ieee.org/abstract/document/6065412) and works best on longer text strings, while `align()` implement the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) and works best on shorter strings. We recommend using the `align_w_anchor()` method on inputs longer than **200 characters**. Both methods share the same function contract and are interchangeable. 

We use [Biopython](https://biopython.org/)'s implementation of the Needleman-Wunsch algorithm for text alignment.
This algorithm is an exhaustive search for all possible candidates with dynamic programming. 
It produces weighted score for each candidate and returns those having the highest score. 
(**NOTE** that multiple candidates can share the same score)

This algorithm has 4 hyperparameters for tuning candidate scores:
1. **Match Reward** - how much the algorithm rewards matching characters
1. **Mismatch Penalty** - how much the algorithm penalizes mismatching characters
1. **Gap Penalty** - how much the algorithm penalizes for creating a gap with a GAP_CHAR (defaults to '@')
1. **Gap Extension Penalty** - how much the algorithm penalizes for extending a gap (ex "@@@@")

You can find the default values for these four parameters as a constant in the package:
1. `genalog.text.alignment.MATCH_REWARD`
1. `genalog.text.alignment.MISMATCH_PENALTY`
1. `genalog.text.alignment.GAP_PENALTY`
1. `genalog.text.alignment.GAP_EXT_PENALTY`
We will demonstrate text alignment here.

In [40]:
gt_txt = "New York is big"
noise_txt = "New Yo rkis"

In [41]:
# RETAS method 
from genalog.text import anchor

# Extra whitespaces are removed
aligned_gt, aligned_noise = anchor.align_w_anchor(gt_txt, noise_txt)
print(f"Aligned ground truth: {aligned_gt}")
print(f"Aligned noise:        {aligned_noise}")

Aligned ground truth: New Yo@rk is big
Aligned noise:        New Yo rk@is@@@@


In [42]:
# Needleman-Wunsch alignment ONLY
from genalog.text import alignment

aligned_gt, aligned_noise = alignment.align(gt_txt, noise_txt)
print(f"Aligned ground truth: {aligned_gt}")
print(f"Aligned noise:        {aligned_noise}")

Aligned ground truth: New Yo@rk is big
Aligned noise:        New Yo rk@is@@@@


In [36]:
from genalog.text import alignment

# Process the aligned strings to find out how the tokens are related
gt_to_noise_mapping, noise_to_gt_mapping = alignment.parse_alignment(aligned_gt, aligned_noise, gap_char="@")
print(f"gt_to_noise: {gt_to_noise_mapping}")
print(f"noise_to_gt: {noise_to_gt_mapping}")

gt_to_noise: [[0], [1, 2], [2], []]
noise_to_gt: [[0], [1], [1, 2], []]


In [25]:
# Format aligned string for better display
print(alignment._format_alignment(aligned_gt, aligned_noise))

New Yo@rk is @ big@
||||||.||.||||||||.
New Yo rk@is @ big 

